In [3]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import gmaps
import os
import requests
import shutil

In [4]:
with open('../resources/key.txt') as f:
    token = f.read()
    
gmaps.configure(api_key = token)

In [5]:
input_folder = '../data/railways/denver_RTD/'
input_file = input_folder+'LightrailLines_Center.shp'

output_folder = '../data/output_images/DenverRTD/'
output_csv = output_folder+'DEN.csv'

update_folder = output_folder+'/set_2/'

### Import Routes

In [4]:
routes = gpd.read_file(input_file)

routes.head()

,OBJECTID,PROJECTCOR,SOURCE,Shape__Len,geometry
0,142,110,A-Line,87291.422422,(LINESTRING (-105.001090582883 39.753148821931...
1,143,112,G-Line,35177.373865,(LINESTRING (-105.137784196155 39.786794639271...
2,144,113,B-Line,7281.026112,(LINESTRING (-105.007403102142 39.806185928394...
3,145,109,L-Line,2816.281729,(LINESTRING (-104.988737522449 39.748187234728...
4,146,113,B-G Lines,20179.760544,(LINESTRING (-104.999945166902 39.754022112124...


In [4]:
# Number of routes
routes.shape

(84, 5)

In [5]:
'''
Get all points
'''

points = []
for i in range(0,len(routes.OBJECTID.tolist())):
    try:
        route = routes.iloc[i]
        g = [m for m in route.geometry]
        for j in range(0,len(g)):
            sec_points = mapping(g[j])["coordinates"]
            for k in range(0,len(sec_points)):
                points.append(sec_points[k])
    except Exception as e:
        pass
#         print("Skipped route number",i,'because',e)

print(len(points),'points')

14230 points


In [ ]:
'''
Get points for 1 or more routes
'''

rows = [0]

points = []
for i in range(0,len(routes.OBJECTID.tolist())):
    if i in rows:
    #     print('Route',i)
        try:
            route = routes.iloc[i]
            g = [m for m in route.geometry]
    #         print('\t',len(g),'sections')
            for j in range(0,len(g)):
    #             print('   section',j)
                sec_points = mapping(g[j])["coordinates"]
                for k in range(0,len(sec_points)):
    #                 print('      point',k)
#                     points.append(sec_points[k])
        except Exception as e:
            print("Skipped route number",i,'because',e)

print(len(points),'points')

In [ ]:
'''
Get points for single route/section
'''

row = 0
section = 0

points = []
try:
    route = routes.iloc[row]
    g = [i for i in route.geometry]
    len(g)

    points = mapping(g[section])["coordinates"]
except Exception as e:
    print("Skipped route number",row,'because',e)
    
print(len(points),'points')

In [6]:
'''
Get subselection of points
'''

route_points = []
for point in range(0,len(points),290):
    tmp = points[point]
    route_points.append(tmp)

print(len(route_points))

50


In [7]:
'''
Create dataframe
'''

df = pd.DataFrame(route_points, columns=['Longitude','Latitude'])

tmp = list(df.Latitude.tolist())
names = []
for i in range(0,len(tmp)):
    name = str(df.iloc[i].Longitude)+'_'+str(df.iloc[i].Latitude)
    names.append(name)

df['Name'] = names

columns = ['Name','Longitude','Latitude']
df= df[columns]


df.head()

,Name,Longitude,Latitude
0,-105.001090582883_39.7531488219317,-105.001091,39.753149
1,-104.971298339626_39.7725748058909,-104.971298,39.772575
2,-104.946459678545_39.7758692108252,-104.946460,39.775869
3,-104.776118952483_39.8273629960361,-104.776119,39.827363
4,-104.865441855775_39.7687013714991,-104.865442,39.768701


In [8]:
'''
Export points
'''

df.to_csv(output_csv,header=True,index=None)

In [6]:
'''
Open csv
'''

df = pd.read_csv(output_csv)
df.head()

,Name,Longitude,Latitude,Catenary
0,-104.731949563008_39.8367816516765,-104.731950,39.836782,1
1,-104.776118952483_39.8273629960361,-104.776119,39.827363,1
2,-104.831955844378_39.7520964362027,-104.831956,39.752096,1
3,-104.865441855775_39.7687013714991,-104.865442,39.768701,1
4,-104.872393928745_39.563547904778,-104.872394,39.563548,1


In [7]:
'''
Plot markers
'''

rows = df.index.tolist()

marker_points = []
for row in rows:
    marker_points.append((df.iloc[row].Latitude,df.iloc[row].Longitude))
    
# Set map centerpoint
coords = marker_points[0]
       
# Define map
m = folium.Map(
    location = coords,
    zoom_start = 13
)
  
# Add points    
for mp in marker_points:
    folium.Marker(mp).add_to(m)
m

In [ ]:
'''
Get satellite preview for image
'''

row = 10

figure_layout = {
    'width': '100%',
    'height': '800px'
}
fig = gmaps.figure(center = (df.iloc[row].Latitude,df.iloc[row].Longitude), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)
fig

In [11]:
'''
Get all images 
'''

root = os.path.dirname(os.path.abspath('Africa.ipynb'))
image_folder = root[:-9]+update_folder[3:]
image_folder

url = 'https://maps.googleapis.com/maps/api/staticmap?'
rows = df.index.tolist()
for i in range(0,2):
    row = df.iloc[i]
    center = str(row.Latitude)+','+str(row.Longitude)
    payload = {
        'center': center, 
        'zoom': 20,
        'size': '640x640',
        'scale': 2,
        'format': 'png32',
        'maptype': 'satellite',
        'key': token
    }
    
    r = requests.get(url,params=payload,stream=True)

    name = df.iloc[i]['Name']
    filename = image_folder+name+'.png'
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 